In [2]:
source("utils/plot.R")

In [3]:
library(ggrepel)

In [12]:
quantile_threshold = 0.99

sf2 <- read_tsv(snakemake@input[[1]], col_types=cols(), skip=2) %>%
    rename(CLR=Likelihood ) %>%
    mutate(
        selcoeff = Alpha/(4*snakemake@params[["popsize"]]),
        Candidate = CLR > quantile(CLR, quantile_threshold)
    )

In [22]:
to_label <- sf2 %>%
    filter(Candidate) %>%
    slice_max(n=5, order_by=CLR)

fig <- ggplot(sf2) +
    geom_point(aes(x=Position/1000, y=CLR, colour=Candidate), size=1) +
    geom_hline(aes(yintercept=quantile(CLR, quantile_threshold)), linetype='dashed', colour='darkgrey') +
    geom_text(aes(x=min(Position/1000), y=quantile(CLR, quantile_threshold)),
              label=paste0(scales::percent(quantile_threshold), ' quantile'),
              fontface='italic', vjust=0, hjust=0, size=3) +
    geom_label_repel(
        data=to_label,
        aes(x=Position/1000, y=CLR, label=paste0('s = ', signif(selcoeff, 2))),
        min.segment.length=0, size=3
    ) +
    labs(
        x='Position (kb)',
        y='CLR'
    ) +
    scale_x_continuous(labels=scales::comma) +
    scale_colour_manual(values=c('lightgrey', 'black')) +
    turkana_theme +
    theme(
        legend.position='none',
        legend.title=element_blank()
    )

turkana_save(snakemake@output[[1]], fig, asp=1.619)